In [1]:
import pandas as pd

In [2]:
KEY = "4355262724562343"


# hex to binary
def hex2bin(hex):
    return bin(int(hex, 16))[2:].zfill(64)


def str2bin(str):
    return "".join(format(ord(i), "08b") for i in str)


KEY = hex2bin(KEY)

In [3]:
def pad_text(text):
    padding_length = 8 - (len(text) % 8)
    padding = chr(padding_length) * padding_length
    return text + padding


def split_into_blocks(text):
    return [text[i : i + 16] for i in range(0, len(text), 16)]


# split into left and right
def split_plain(plaintext):
    return plaintext[:32], plaintext[32:]

In [4]:
# initial permutation
def initial_permutation(text):
    key = pd.read_csv("initial_permutation.txt", header=None).to_numpy().flatten()
    return "".join(text[i - 1] for i in key)


def final_permutation(text):
    key = pd.read_csv("final_permutation.txt", header=None).to_numpy().flatten()
    return "".join(text[i - 1] for i in key)

In [5]:
# expansion p-box
def expansion_pbox(right):
    key = pd.read_csv("expansion_p_box.txt", header=None).to_numpy().flatten()
    return "".join(right[i - 1] for i in key)


def straight_pbox(text):
    key = pd.read_csv("straight_pbox.txt", header=None).to_numpy().flatten()
    return "".join(text[i - 1] for i in key)


# compression p-box for key compression
def compression_pbox(key):
    key_compression = (
        pd.read_csv("key_compression.txt", header=None).to_numpy().flatten()
    )
    return "".join(key[i - 1] for i in key_compression)


# key parity drop
def key_parity_drop(key):
    key_drop = pd.read_csv("parity_bit_drop.txt", header=None).to_numpy().flatten()
    return "".join(key[i - 1] for i in key_drop)

In [6]:
# Key generation
def key_generation(key):
    # key parity drop
    key = key_parity_drop(key)

    # split key into left and right
    left, right = key[:28], key[28:]

    # shift left
    left = left[1:] + left[:1]
    right = right[1:] + right[:1]

    # compression p-box
    return compression_pbox(left + right)

In [7]:
first_key = key_generation(KEY)
print(first_key, len(first_key))

110000001011010011000100100000110010001100010011 48


In [8]:
# s-boxes
import pickle


def s_box(input_bit_string):
    with open("sbox.pkl", "rb") as f:
        sbox = pickle.load(f)

    bits_per_sbox = [
        input_bit_string[i : i + 6] for i in range(0, len(input_bit_string), 6)
    ]

    output = ""
    for ind, s in enumerate(sbox):
        row = int(bits_per_sbox[ind][0] + bits_per_sbox[ind][5], 2)
        col = int(bits_per_sbox[ind][1:5], 2)
        out = s[row][col]
        output += format(out, "04b")

    return output

In [9]:
# final permutation
def rev_final_permutation(text):
    final_perm = pd.read_csv("final_permutation.txt", header=None).to_numpy().flatten()
    ls = [0 for __ in range(len(final_perm))]

    for ind, val in enumerate(final_perm):
        ls[val - 1] = text[ind]

    return "".join(ls)

In [10]:
# xor left and all_left[i]
def xor(l, r):
    return "".join(str(int(l[i]) ^ int(r[i])) for i in range(len(l)))

In [11]:
# from initial permutation to straight p-box
def from_initial_permutation_to_straight_pbox(txt_paded):
    text_in = str2bin(txt_paded)

    in_per = initial_permutation(text_in)

    # split into left and right
    left, right = split_plain(in_per)

    # expansion p-box
    expansion = expansion_pbox(right)

    # xor expand_right and first_key
    xored_right_key = xor(expansion, first_key)

    # s-boxes
    after_sbox = s_box(xored_right_key)

    return after_sbox, left

In [12]:
def from_cipher_to_straight_pbox(text, left_in):
    text_out = hex2bin(text)

    # final permutation
    after_final_perm = rev_final_permutation(text_out)

    # split into left and right
    left, right = split_plain(after_final_perm)

    # xor left and all_left
    after_straight_pbox = xor(left, left_in)

    return after_straight_pbox

In [13]:
input_pairs = []
with open("known_plaintext.txt", "r") as f:
    data = f.read()
    input_pairs = data.split(", ")

output_pairs = []
with open("known_ciphertext.txt", "r") as f:
    data = f.read()
    output_pairs = data.split(", ")

tmp_in, tmp_out = [], []
for i in range(len(input_pairs)):
    padded = pad_text(input_pairs[i])
    if len(padded) > 8:
        input_pairs[i] = padded[:8]
        tmp_in.append(padded[8:])
        tmp_str = output_pairs[i]
        output_pairs[i] = tmp_str[:16]
        tmp_out.append(tmp_str[16:])
    else:
        input_pairs[i] = padded

input_pairs.extend(tmp_in)
output_pairs.extend(tmp_out)

print(input_pairs)
print(output_pairs)

['kootahe\x01', 'Zendegi\x01', 'Edame\x03\x03\x03', 'Dare\x04\x04\x04\x04', 'JolotYe\x01', 'Daame\x03\x03\x03', 'DaemKe\x02\x02', 'Mioftan\x01', 'Toosh\x03\x03\x03', 'HattaMo\x01', 'khayeSa\x01', '05753jj\x01', '==j95697', '\x08\x08\x08\x08\x08\x08\x08\x08']
['6E2F7B25307C3144', 'CF646E7170632D45', 'D070257820560746', '5574223505051150', 'DB2E393F61586144', 'D175257820560746', 'D135603D1A705746', 'D83C6F7321752A54', '413A2B666D024747', '5974216034186B44', 'EA29302D74463545', 'B1203330722B7A04', '38693B6824232D23', '1D1C0D0C4959590D']


In [14]:
def find_straight_pbox_pattern(inputt, output):
    lst = []
    for i in range(len(output)):
        ls = []
        for j in range(len(inputt)):
            if output[i] == inputt[j]:
                ls.append(j + 1)
        lst.append(ls)
    return lst

In [15]:
res = []
for input_p, output_p in zip(input_pairs, output_pairs):
    start, left = from_initial_permutation_to_straight_pbox(input_p)
    final = from_cipher_to_straight_pbox(output_p, left)
    print(start)
    print(final)
    print("---------------------------------------")
    res.append(find_straight_pbox_pattern(start, final))

00111111111000100100101101111001
11011010011111001110010110011001
---------------------------------------
00111111111001011010000111000000
11000001000110011111100100011011
---------------------------------------
11111111101101011011001100110101
10110101001010111111111110111001
---------------------------------------
11111111000110110100000010001110
10001100110011111100001001110011
---------------------------------------
00111111010101011100110010101101
10001111000111011101010010110111
---------------------------------------
11111111101101011011001100110101
10110101001010111111111110111001
---------------------------------------
11111111110010111000111101111100
11011011011110111110011101110101
---------------------------------------
11111111111001011010101111010011
11010011101110111111101110011011
---------------------------------------
11111111101101010010100011100100
11000110000010111101111100111011
---------------------------------------
11111111010101010111011110110101
1011010000111

In [18]:
with open("spb_temp.txt", "w") as f:
    for i in range(32):
        d = []
        for j in range(len(res)):
            d.append(res[j][i])
        f.write(" ".join(map(str, set.intersection(*map(set, d)))))
        f.write("\n")

In [19]:
cphier_test = "59346E29456A723B62354B61756D44257871650320277C741D1C0D0C4959590D"
blocks = split_into_blocks(cphier_test)
blocks

['59346E29456A723B',
 '62354B61756D4425',
 '7871650320277C74',
 '1D1C0D0C4959590D']

In [20]:
def dec(txt):
    text_in = hex2bin(txt)

    in_per = initial_permutation(text_in)

    # split into left and right
    left, right = split_plain(in_per)

    # expansion p-box
    expansion = expansion_pbox(right)

    # xor expand_right and first_key
    xored_right_key = xor(expansion, first_key)

    # s-boxes
    after_sbox = s_box(xored_right_key)

    spb = straight_pbox(after_sbox)

    return final_permutation(xor(left, spb) + right)

In [22]:
for b in blocks:
    binary_string = dec(b)
    output_string = "".join(
        chr(int(binary_string[i * 8 : i * 8 + 8], 2))
        for i in range(len(binary_string) // 8)
    )
    print(output_string, end="")

HajiDorostZadiDametGar